In [20]:
# HELPER FUNCTIONS

import collections

def get_mean_metrics(models, metric_type):
    mean_acc_fedavg = []
    for model in models["result"]:
        model_id = model["model"]
        validations = client.list_validations(modelId=model_id)

        acc = []
        for _, validation in validations.items(): 
            metrics = json.loads(validation['data'])
            if metric_type in metrics:
                acc.append(metrics[metric_type])

        if acc:
            mean_acc_fedavg.append(np.mean(acc))

    return np.min(mean_acc_fedavg) if 'loss' in metric_type else np.max(mean_acc_fedavg)

def get_baseline_metrics(baseline_session_id):
    models = client.list_models(baseline_session_id)

    baseline_metrics = {
        'training_loss': get_mean_metrics(models, 'training_loss'),
        'training_accuracy': get_mean_metrics(models, 'training_accuracy'),
        'test_loss': get_mean_metrics(models, 'test_loss'),
        'test_accuracy': get_mean_metrics(models, 'test_accuracy')
    }

    return baseline_metrics

In [21]:
from fedn import APIClient
import time
import uuid
import json
import matplotlib.pyplot as plt
import numpy as np
import collections

import sys
sys.path.append('/home/ubuntu/fedn-attack-sim-uu/examples/mnist-pytorch')

from combiner_config import COMBINER_IP

DISCOVER_HOST = COMBINER_IP
DISCOVER_PORT = 8092
client = APIClient(DISCOVER_HOST, DISCOVER_PORT)

In [22]:
[session['session_id'] for session in client.list_sessions()['result']]

['BACKDOOR_TEST_04',
 'BACKDOOR_TEST_03',
 'BACKDOOR_TEST_02',
 'BACKDOOR_TEST_01',
 'MNIST_GBP100_25R_16_04',
 'MNIST_GBP100_25R_18_02',
 'MNIST_GBP100_25R_19_01',
 'MNIST_GBP010_25R_16_04',
 'MNIST_GBP010_25R_18_02',
 'MNIST_GBP010_25R_19_01',
 'MNIST_LFB_25R_16_04_V2',
 'MNIST_LFB_25R_18_02_V2',
 'MNIST_LFB_25R_19_01_V2',
 'TEST_5_WITH_EPOCH_01',
 'TEST_EPOCH_04',
 'TEST_EPOCH_03',
 'TEST_EPOCH_02',
 'TEST_EPOCH_01',
 'MNIST_GBM001_25R_18_02',
 'MNIST_GBM001_25R_19_01',
 'MNIST_BASE_25R_20_00']

In [23]:
session_id = input("Provide a session_id: ")

Provide a session_id:  BACKDOOR_TEST_04


In [24]:
baseline_metrics = get_baseline_metrics(input("Provide baseline session_id: "))

Provide baseline session_id:  MNIST_BASE_25R_20_00


In [25]:
models = client.list_models(session_id)

num_rounds = models['count']
# num_rounds

# Provides a model trail for a given session (in FILO) --> need to be reversed
model_trail = models['result']
model_trail.reverse()
# model_trail

In [38]:
client.list_validations(modelId = model_trail[20]['model'])

{'663757fd981afd046cebee22': {'data': '{"training_loss": 0.611663281917572, "training_accuracy": 0.8330000042915344, "test_loss": 0.31753280758857727, "test_accuracy": 0.949999988079071, "train_target_misclassification": 0.21967213114754103, "test_target_misclassification": 0.08333333333333337, "train_target_prediction_dist": [1, 12, 8, 17, 3, 6, 4, 6, 238, 10], "test_target_prediction_dist": [1, 1, 0, 0, 0, 1, 1, 0, 44, 0]}',
  'meta': '',
  'model_id': 'a727e6f5-6c3e-4b53-a6e4-5eacb3fc880b',
  'receiver': {'name': 'combiner', 'role': 'COMBINER'},
  'sender': {'name': 'malicious_client2', 'role': 'WORKER'},
  'timestamp': '2024-05-05T09:57:17.572398Z'},
 '663757fd981afd046cebee23': {'data': '{"training_loss": 0.5198166370391846, "training_accuracy": 0.8823333382606506, "test_loss": 0.6905912160873413, "test_accuracy": 0.8220000267028809, "train_target_misclassification": 0.1582491582491582, "test_target_misclassification": 0.20833333333333337, "train_target_prediction_dist": [7, 10, 1

In [39]:
json.loads(client.list_validations(modelId = model_trail[20]['model'])['663757fd981afd046cebee22']['data'])

{'training_loss': 0.611663281917572,
 'training_accuracy': 0.8330000042915344,
 'test_loss': 0.31753280758857727,
 'test_accuracy': 0.949999988079071,
 'train_target_misclassification': 0.21967213114754103,
 'test_target_misclassification': 0.08333333333333337,
 'train_target_prediction_dist': [1, 12, 8, 17, 3, 6, 4, 6, 238, 10],
 'test_target_prediction_dist': [1, 1, 0, 0, 0, 1, 1, 0, 44, 0]}

In [55]:
for key in client.list_validations(modelId = model_trail[24]['model']).keys():
    print(json.loads(client.list_validations(modelId = model_trail[24]['model'])[key]['data'])['test_target_prediction_dist'])

[1, 2, 2, 1, 0, 3, 0, 2, 33, 0]
[1, 1, 0, 0, 0, 1, 1, 0, 44, 0]
[0, 1, 0, 1, 0, 2, 3, 0, 31, 4]
[1, 0, 0, 1, 1, 3, 0, 2, 30, 2]
[0, 0, 1, 0, 0, 1, 0, 1, 44, 2]
[1, 0, 0, 0, 3, 4, 2, 0, 34, 1]
[0, 3, 1, 3, 2, 1, 0, 0, 36, 2]
[0, 0, 0, 6, 0, 2, 0, 3, 33, 1]
[0, 1, 0, 1, 2, 2, 0, 1, 40, 3]
[0, 0, 1, 1, 5, 3, 1, 0, 37, 0]
[1, 1, 1, 4, 3, 0, 2, 2, 36, 3]
[1, 0, 0, 3, 1, 3, 1, 1, 41, 1]
[0, 0, 0, 2, 0, 0, 1, 0, 47, 3]
[0, 0, 0, 1, 1, 2, 0, 0, 41, 1]
[1, 1, 0, 2, 0, 5, 0, 0, 46, 1]
[0, 2, 1, 4, 0, 5, 0, 0, 40, 1]
[1, 2, 1, 2, 2, 3, 1, 0, 37, 2]
[0, 0, 1, 4, 1, 3, 2, 1, 34, 3]
[1, 3, 2, 3, 0, 3, 1, 2, 36, 1]
[2, 3, 0, 6, 2, 1, 0, 1, 32, 3]


In [47]:
np.sum([[1, 12, 8, 17, 3, 6, 4, 6, 238, 10], [1, 1, 0, 0, 0, 1, 1, 0, 44, 0], [1, 12, 8, 17, 3, 6, 4, 6, 238, 10]], axis=0).tolist()

[3, 25, 16, 34, 6, 13, 9, 12, 520, 20]

In [ ]:
validations = []

for model_instance in model_trail:
    worker_results = client.list_validations(modelId=model_instance['model'])
    result_ids = [result_id for result_id in worker_results]
    for result_id in result_ids:
        worker_result = worker_results.get(result_id)
        client_type = 'benign'
        
        if 'malicious' in worker_result['sender']['name']:
            client_type = 'malicious'
            
        validation = {
            'model_committed_at': model_instance['committed_at'],
            'session_id': model_instance['session_id'],
            'model': model_instance['model'],
            'client_name': worker_result['sender']['name'],
            'client_role': worker_result['sender']['role'],
            'client_type': client_type,
            'training_loss': json.loads(worker_result['data'])['training_loss'],
            'training_accuracy': json.loads(worker_result['data'])['training_accuracy'],
            'test_loss': json.loads(worker_result['data'])['test_loss'],
            'test_accuracy': json.loads(worker_result['data'])['test_accuracy']
        }
        
        validations.append(validation)
        